## ETF DATA

섹터별 퍼포먼스 heatmap 만들기

In [1]:
import datetime
import os
import time
from dotenv import load_dotenv
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from pymongo.mongo_client import MongoClient

load_dotenv()

True

In [2]:
url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Content-Length": "101",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Cookie": "",
    "Host": "data.krx.co.kr",
    "Origin": "http://data.krx.co.kr",
    "Pragma": "no-cache",
    "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201030101",
    "User-Agent": os.getenv("USER_AGENT"),
    "X-Requested-With": "XMLHttpRequest"
}
data = {
    "bld": "dbms/MDC/STAT/standard/MDCSTAT04301",
    "locale": "ko_KR",  
    "trdDd": "20250711",
    "share": "1",
    "money": "1",
    "csvxls_isNo": "false"
}

req = requests.post(url, headers=headers, data=data)

In [3]:
data = req.json()['output']
etf_df = pd.DataFrame(data)[["ISU_ABBRV", "IDX_IND_NM", "ISU_SRT_CD","ISU_CD", "MKTCAP"]]
etf_df["MKTCAP"] = etf_df["MKTCAP"].str.replace(",", "").astype(float)
etf_df = etf_df.loc[etf_df["MKTCAP"].nlargest(600).index]

In [4]:
mask = (
    etf_df["ISU_ABBRV"].str.contains("금리") |
    etf_df["ISU_ABBRV"].str.contains("크레딧") |
    etf_df["ISU_ABBRV"].str.contains("MMF") |
    etf_df["ISU_ABBRV"].str.contains("머니마켓") |
    etf_df["ISU_ABBRV"].str.contains("채권") |
    etf_df["ISU_ABBRV"].str.contains("회사채")
)
etf_df = etf_df[~mask]

In [5]:
uri = os.getenv("MONGODB_URI")
client = MongoClient(uri)

# 새로운 콜렉션 생성 (예: etf_krx_ohlcv)
db = client['quant']
collection = db['krx_etf']

In [6]:

url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

# 오늘 날짜와 일주일 전 날짜 계산
today = datetime.datetime.today()
one_week_ago = today - datetime.timedelta(days=3)
strtDd = one_week_ago.strftime("%Y%m%d")
endDd = today.strftime("%Y%m%d")

data_dict = {
    "bld": "dbms/MDC/STAT/standard/MDCSTAT04501",
    "locale": "ko_KR",
    "param1isuCd_finder_secuprodisu1_1": "",
    "strtDd": strtDd,
    "endDd": endDd,
    "share": "1", # 조회기간 최대 2년
    "money": "1",
    "csvxls_isNo": "false"
}
headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Content-Length": "290",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Cookie": "",
    "Host": "data.krx.co.kr",
    "Origin": "http://data.krx.co.kr",
    "Pragma": "no-cache",
    "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201030101",
    "User-Agent": os.getenv("USER_AGENT"),
    "X-Requested-With": "XMLHttpRequest"
}
l = []
for idx, row in tqdm(etf_df.iterrows(), total=len(etf_df)):
    data_dict["isuCd"] = row["ISU_CD"]
    req = requests.post(url, data=data_dict, headers=headers)
    df = pd.DataFrame(req.json()['output'])
    
    if 'TRD_DD' in df.columns:
        df['TRD_DD'] = pd.to_datetime(df['TRD_DD'], errors='coerce')

    for col in df.columns:
        if df[col].dtype == object:
            try:
                df[col] = df[col].str.replace(',', '', regex=False)
                df[col] = pd.to_numeric(df[col])
            except Exception:
                pass
    l.append(df)
    
    # MongoDB에 적재
    # 각 ETF별로, 해당 ETF의 ISU_CD와 이름을 document에 추가
    if not df.empty:
        df['ISU_CD'] = row['ISU_CD']
        df['ISU_ABBRV'] = row['ISU_ABBRV']
        # 이미 DB에 존재하는 데이터는 제외
        # TRD_DD(거래일), ISU_CD(종목코드)로 중복 체크
        insert_records = []
        for record in df.to_dict(orient='records'):
            # TRD_DD가 NaT일 수 있으니 문자열로 변환
            trd_dd = record.get('TRD_DD')
            if pd.isnull(trd_dd):
                continue

            if isinstance(trd_dd, pd.Timestamp):
                trd_dd_mongo = trd_dd.to_pydatetime()
            else:
                try:
                    trd_dd_mongo = pd.to_datetime(trd_dd).to_pydatetime()
                except Exception:
                    continue
            exists = collection.count_documents({
                "TRD_DD": trd_dd_mongo,
                "ISU_CD": record["ISU_CD"]
            }, limit=1)

            if not exists:
                # DB에 없는 데이터만 추가
                record['TRD_DD'] = trd_dd_mongo
                insert_records.append(record)
                
        if insert_records:
            try:
                collection.insert_many(insert_records)
            except Exception as e:
                print(f"MongoDB 적재 에러: {e}")
    time.sleep(np.random.randint(0.5, 2))

  0%|          | 0/506 [00:00<?, ?it/s]

100%|██████████| 506/506 [12:53<00:00,  1.53s/it]
